In [46]:
import os
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu

osl = os.listdir
ospj = os.path.join

### get relevant runs data given filters of interest

In [47]:
import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key=alphanum_key)

In [62]:
"""
runs_dir = "/home/derposoft/ray_results"
runs_filters = [
    "fcscout_baseline200h_1r2b_SEED",
    "gnnscout_200h2.2l_1r2b_SEED",
    "gatscout_200h2.2l_1r2b_SEED"
]
runs_filters = [
    "fcskirmish_5hp_baseline100_50_PAPER_SEED",
    "gcnskirmish_5hp_baseline100_50_3.4_PAPER_SEED",
    "gatLNskirmish_5hp_GLOBALbaseline100_50_3.4_PAPER_SEED",
    "gcnLNskirmish_5hp_baseline100_50_3.4_PAPER_SEED",
]
runs_filters = [
    "fcskirmish_5hp_baseline100_50_100ksteps_SEED",
    "gcnskirmish_5hp_baseline100_50_2.4_PAPER_SEED",
]
runs_dirs = [
    "/home/derposoft/Documents/tb_logs/40k_runs/fcskirmish_100_50",
    "/home/derposoft/Documents/tb_logs/40k_runs/gcnskirmish_100_50_3.4",
]
runs_filters = [
    "fcskirmish_5hp_baseline100_50_PAPER_SEED",
    "gcnskirmish_5hp_baseline100_50_3.4_PAPER_SEED",
]
"""
runs_dirs = [
    "/home/derposoft/Documents/tb_logs/seeded_runs/",
    "/home/derposoft/Documents/tb_logs/tb_logs.2022-12-18_paper/"
]
runs_filters_5hp = [
    "baseline_SEED",
    "hybridGAT_5hp_globalemb_NOOPT_SEED",
    "hybridGAT_5hp_hybridemb_NOOPT_SEED",
    "hybridGAT_5hp_localemb_NOOPT_SEED",
    "hybridGCN_5hp_globalemb_NOOPT_SEED",
    "hybridGCN_5hp_hybridemb_NOOPT_SEED",
    "hybridGCN_5hp_localemb_NOOPT_SEED",
    "hybridGT_5hp_globalemb_NOOPT_SEED",
    "hybridGT_5hp_hybridemb_NOOPT_SEED",
    "hybridGT_5hp_localemb_NOOPT_SEED",
]
runs_filters_2hp = [
    "baseline_2hp_NOOPT_SEED",
    "hybridGAT_2hp_localemb_NOOPT_SEED",
    "hybridGCN_2hp_localemb_NOOPT_SEED",
    "hybridGT_2hp_localemb_NOOPT_SEED",
]
runs_filters_20hp = [
    "baseline_20hp_NOOPT_SEED",
    "hybridGAT_20hp_localemb_NOOPT_SEED",
    "hybridGCN_20hp_localemb_NOOPT_SEED",
    "hybridGT_20hp_localemb_NOOPT_SEED",
]
runs_filters_5hp_2v2 = [
    "baseline_5hp_NOOPT_2r2b_SEED",
    "hybridGAT_5hp_localemb_NOOPT_2r2b_SEED",
    "hybridGCN_5hp_localemb_NOOPT_2r2b_SEED",
    "hybridGT_5hp_localemb_NOOPT_2r2b_SEED",
]
runs_filters = runs_filters_5hp_2v2
runs = sum(([ospj(runs_dir, x) for x in osl(runs_dir)] for runs_dir in runs_dirs), [])
tot_steps_len = 200
runs = natural_sort(runs)

In [63]:
all_filters_runs = [
    [run for run in runs if runs_filter in run]
    for runs_filter in runs_filters
]

In [65]:
run_data_file = "progress.csv"
run_data_col = "episode_reward_mean"
run_data_col = "evaluation/episode_reward_mean"
run_step_col = "timesteps_total"
all_runs_data = []
for filter_runs in all_filters_runs:
    curr_run_data = []
    for run in filter_runs:
        datafile = pd.read_csv(ospj(run, run_data_file))
        curr_run_data.append(datafile[run_data_col])
    all_runs_data.append(curr_run_data)
runs_steps = [
    pd.read_csv(ospj(run, run_data_file))[run_step_col]
    for run in runs
]

### given data, run "map" on data to get values suitable for analysis

In [66]:
# finds the first training step when reward exceeds a given value ge
def first_ep_to_val(runs, steps, ge=30):
    first_eps = []
    for run, steps in zip(runs, steps):
        found = False
        for step, val in zip(steps, run):
            if val >= ge:
                first_eps.append(step)
                found = True
                break
        if not found:
            first_eps.append(-1)
    return first_eps

# finds max avg reward
def max_avg_reward(runs, steps):
    max_vals = []
    for run in runs:
        if len(run) < tot_steps_len: continue
        max_vals.append(max(run))
    return max_vals

# returns last reward
def last_ep_reward(runs, steps):
    last_vals = []
    for run in runs:
        if len(run) < tot_steps_len: continue
        last_vals.append(run.tolist()[-1])
    return last_vals

In [67]:
first_ep_to_30 = [
    first_ep_to_val(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]
max_reward = [
    max_avg_reward(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]
last_reward = [
    last_ep_reward(filter_runs_data, runs_steps)
    for filter_runs_data in all_runs_data
]

### given "map" values, run "reduce" to aggregate values and show significance

In [68]:
# pretty print a dictionary
def pretty_print(d, tabs=0, tabsize=4):
    for k in d:
        v = d[k]
        nspaces = " " * tabsize * tabs
        if type(v) == dict:
            print(f"{nspaces}{k}:")
            pretty_print(v, tabs+1, tabsize)
        else:
            print(f"{nspaces}{k}: {v}")

# get stats for mapped values for a given experiment
def get_stats(all_runs_vals, firstn=-1):
    stats = {}
    #baselines = all_runs_vals[0]
    baselines = all_runs_vals[0]
    for filter, vals in zip(runs_filters, all_runs_vals):
        if firstn > 0: vals = vals[:firstn]
        mu = np.mean(vals)
        sigma = np.std(vals)
        ci95z = 1.96
        ttest_results = ttest_ind(baselines, vals, alternative="less", equal_var=False)
        stats[filter] = {
            "str": f"{mu}+/-{ci95z*sigma} -- p={ttest_results.pvalue}",
            #"mean": f"{mu}+/-{ci95z*sigma}",
            #"median": np.median(vals),
            #"std": sigma,
            #"ci": [mu-ci95z*sigma, mu+ci95z*sigma],
            #"n": len(vals),
            #"mannwu": mannwhitneyu(baselines, vals, alternative="less"),
            #"ttest_less": ttest_ind(baselines, vals, alternative="less", equal_var=False),
            #"ttest_more": ttest_ind(baselines, vals, alternative="greater", equal_var=False),
        }
    return stats

In [69]:
firstn = -1
first_ep_to_30_stats = get_stats(first_ep_to_30, firstn=firstn)
max_reward_stats = get_stats(max_reward, firstn=firstn)
last_reward_stats = get_stats(last_reward, firstn=firstn)

In [70]:
def print_stat(stats, name=""):
    assert name != ""
    print(name+" stats:")
    pretty_print(stats)
    print()

print_stat(max_reward_stats, "max reward")
print_stat(first_ep_to_30_stats, "first ep to 30")
print_stat(last_reward_stats, "last reward")


max reward stats:
baseline_5hp_NOOPT_2r2b_SEED:
    str: 127.14000000000001+/-28.442051582823627 -- p=0.5
hybridGAT_5hp_localemb_NOOPT_2r2b_SEED:
    str: 132.76+/-28.99160022903185 -- p=0.2132386035153495
hybridGCN_5hp_localemb_NOOPT_2r2b_SEED:
    str: 184.45+/-33.11776568550481 -- p=2.3561312752656607e-07
hybridGT_5hp_localemb_NOOPT_2r2b_SEED:
    str: 133.41+/-31.857260218669147 -- p=0.19974155040735814

first ep to 30 stats:
baseline_5hp_NOOPT_2r2b_SEED:
    str: 800.0+/-1288.2434552521506 -- p=0.5
hybridGAT_5hp_localemb_NOOPT_2r2b_SEED:
    str: 460.0+/-841.6596937004884 -- p=0.8935613875672169
hybridGCN_5hp_localemb_NOOPT_2r2b_SEED:
    str: 600.0+/-960.1999791710058 -- p=0.7628016914841806
hybridGT_5hp_localemb_NOOPT_2r2b_SEED:
    str: 620.0+/-901.6 -- p=0.7447906964313427

last reward stats:
baseline_5hp_NOOPT_2r2b_SEED:
    str: 91.11000000000001+/-37.809050279529636 -- p=0.5
hybridGAT_5hp_localemb_NOOPT_2r2b_SEED:
    str: 95.77+/-46.297842854284255 -- p=0.3261836422017079
